In [21]:
using DataFrames, CSV, Plots, JuMP, GLPK, Statistics, StatsPlots,Gurobi
import Dates
ENV["ROWS"]=120

120

### TMY Calculation

In [22]:
timeseries = CSV.read("Data/time_series_60min_singleindex.csv", DataFrame)
columns = names(timeseries)
to_keep = columns[occursin.("GB",columns) .& occursin.("profile",columns) .& (occursin.("wind",columns) .| occursin.("solar",columns))]
to_keep_ts = vcat(columns[1], to_keep)

timeseries = timeseries[!,to_keep_ts]
CSV.write("Data/timeseries_uk_erv.csv", timeseries)

#We'll create a TMY with the Solar data, Onshore wind data and offshore data
timeseries[!,"timestamp"] = Dates.DateTime.(timeseries[!,"utc_timestamp"],"yyyy-mm-ddTHH:MM:SSZ");
# We filter the data so we don't have missing values anywhere
timeseries_dict = Dict()
#for column in to_keep
#    timeseries_dict[column] = timeseries[.!ismissing.(timeseries[!,column]),["timestamp",column]]
#end
for column in to_keep
    timeseries_dict[column] = timeseries[!,["timestamp",column]]
    #timeseries_dict[column] = timeseries_dict[column][.!ismissing.(timeseries_dict[column][!,column]),:]
end

# From each date, we save the year, month, day and hour, as the data is hourly
for column in to_keep
    timeseries_dict[column][!,"year"] = Dates.year.(timeseries_dict[column][!,"timestamp"])
    timeseries_dict[column][!,"month"] = Dates.month.(timeseries_dict[column][!,"timestamp"])
    timeseries_dict[column][!,"day"] = Dates.day.(timeseries_dict[column][!,"timestamp"])
    timeseries_dict[column][!,"hour"] = Dates.hour.(timeseries_dict[column][!,"timestamp"])
    timeseries_dict[column][!,"missing"] = ismissing.(timeseries_dict[column][!,column])
end

long_term_data = Dict()
long_term_year = Dict()
# Now, we group by month and day so we can get the average capacity factor
for element in timeseries_dict
    name, df = element
    df = element[2];
    gdf = groupby(df, [:month, :year])
    gdf = combine(gdf, name .=> mean => name)
    long_term_data[name] = gdf

    df = gdf;
    gdf = groupby(df, :month)
    gdf = combine(gdf, name .=> mean => name)
    long_term_year[name] = gdf
end

#upper, lower = max(long_term_data["GB_GBN_solar_profile"][!,:year]...),min(long_term_data["GB_GBN_solar_profile"][!,:year]...)
#upper, lower = 2018, 2015
#years = upper-lower;


In [23]:
u = timeseries_dict["GB_GBN_solar_profile"]

Row,timestamp,GB_GBN_solar_profile,year,month,day,hour,missing
,DateTime,Float64?,Int64,Int64,Int64,Int64,Bool
1,2014-12-31T23:00:00,missing,2014,12,31,23,true
2,2015-01-01T00:00:00,missing,2015,1,1,0,true
3,2015-01-01T01:00:00,missing,2015,1,1,1,true
4,2015-01-01T02:00:00,missing,2015,1,1,2,true
5,2015-01-01T03:00:00,missing,2015,1,1,3,true
6,2015-01-01T04:00:00,missing,2015,1,1,4,true
7,2015-01-01T05:00:00,missing,2015,1,1,5,true
8,2015-01-01T06:00:00,missing,2015,1,1,6,true
9,2015-01-01T07:00:00,missing,2015,1,1,7,true


In [24]:
monthly_avg = Dict()
for element in timeseries_dict
    name, df = element
    avgs = []
    valid_years_list =[]
    for k in 1:12
        # Determine month with missing data
        monthly_df = groupby(df[df.month.==k,:],:year)
        monthly_df = combine(monthly_df, "missing" .=> sum => "missing")
        valid_years = monthly_df[monthly_df[!,"missing"] .== 0,:year]
        avg = mean(df[(df.month.==k) .& (df.year .∈ Ref(valid_years)),name])
        push!(avgs,avg)
        push!(valid_years_list,valid_years)
    end
    monthly_avg[name] = avgs
    monthly_avg[name*"_years"] = valid_years_list
end

avg_data = Dict()
for element in timeseries_dict
    name, df = element
    avg_matrix = []
    for k in 1:12
        monthly_df = groupby(df[df.month.==k,:],:year)
        monthly_df = combine(monthly_df, "missing" .=> sum => "missing")
        valid_years = monthly_df[monthly_df[!,"missing"] .== 0,:year]
        yearly_means = []
        for j in valid_years
            yearly_mean = mean(df[(df.month.==k) .& (df.year.==j),name])
            push!(yearly_means,yearly_mean)
        end
        push!(avg_matrix,yearly_means)
    end
    avg_data[name] = avg_matrix
end


In [25]:
monthly_avg["GB_GBN_solar_profile_years"]

12-element Vector{Any}:
 [2016, 2017, 2018, 2019]
 [2015, 2016, 2017, 2018, 2019]
 [2015, 2016, 2018, 2019]
 [2015, 2016, 2017, 2018, 2019]
 [2016, 2017, 2018, 2019]
 [2015, 2016, 2017, 2018, 2019]
 [2015, 2016, 2017, 2018, 2019]
 [2015, 2016, 2018, 2019]
 [2015, 2016, 2018, 2019]
 [2015, 2016, 2017, 2018, 2019]
 [2015, 2016, 2018, 2019]
 [2015, 2016, 2017, 2018]

In [26]:
TMYS = Dict()
for element in avg_data
    name, matrix = element
    best_value = monthly_avg[name]
    df = timeseries_dict[name][!,[name, "month", "year","day","hour"]]
    TMY = DataFrame([[],[],[],[],[]], [name,"year","month","day","hour"])
    best_years = []
    years_lookup = monthly_avg[name*"_years"]
    for k in 1:12
        error = abs.(matrix[k].-best_value[k])
        best_year_index = findall(x->x==min(error...), error)
        best_year = years_lookup[k][best_year_index[1]]
        best_year_df = df[(df.month.==k) .& (df.year.==best_year),[name,"year","month","day","hour"]]
        TMY = vcat(TMY,best_year_df)
        push!(best_years,best_year)
    end
    # Set every timestamp to 2023
    TMY[!,"timestamp"] = Dates.DateTime.("2023".*"-".*string.(TMY[!,"month"]).*"-".*string.(TMY[!,"day"]).*"T".*string.(TMY[!,"hour"]).*":00:00","yyyy-mm-ddTHH:MM:SS");
    TMYS[name] = TMY[!,["timestamp",name,"month"]]
    #CSV.write("TMY_"*name*".csv", TMY)
end



In [27]:
toCSVdict = []
for element in TMYS
    name, df = element
    push!(toCSVdict,df[!,["timestamp",name]])
end

innerjoin(toCSVdict..., on=:timestamp) |> CSV.write("Data/TMY_uk_erv.csv")

"Data/TMY_uk_erv.csv"

In [28]:
plots = []
for element in long_term_year
    name, df = element
    df_tmy = TMYS[name]
    df_raw = timeseries_dict[name]
    df_tmy = groupby(df_tmy, :month)
    df_tmy = combine(df_tmy, name .=> mean => name)
    display_parts = titlecase.(split(name,"_"));
    display_name = join(display_parts[3:end]," ");
    plot(df[!,:month],df[!,name],label="Long Term",color="orange",legend=:topleft, title=display_name)
    for k in lower:upper
        df_year = df_raw[df_raw.year.==k,:]
        df_year = groupby(df_year, :month)
        df_year = combine(df_year, name .=> mean => name)
        plot!(df_year[!,:month],df_year[!,name],label="", color="blue", alpha=0.25)
    end
    p=plot!(df_tmy[!,:month],df_tmy[!,name],label="TMY", color="green")
    push!(plots,p)
end
plot(plots..., layout=(4,1), size=(1000,1000), yformatter=y->string(trunc(Int,100*y))*"%", show=true)
xticks!(1:12,["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"])
#savefig("Results/TMY_comparison.png")

UndefVarError: UndefVarError: `lower` not defined

In [29]:
groupby(TMYS["GB_GBN_solar_profile"],:month)

Row,timestamp,GB_GBN_solar_profile,month
,DateTime,Any,Any
1,2023-01-01T00:00:00,0.0,1
2,2023-01-01T01:00:00,0.0,1
3,2023-01-01T02:00:00,0.0,1
4,2023-01-01T03:00:00,0.0,1
5,2023-01-01T04:00:00,0.0,1
6,2023-01-01T05:00:00,0.0,1
7,2023-01-01T06:00:00,0.0,1
8,2023-01-01T07:00:00,0.0,1
9,2023-01-01T08:00:00,0.0067,1


### Variable Definitions and Model

In [30]:
generators = CSV.read("Data/CostsData.csv",DataFrame);
G = generators.Gen;

# Separate the generators into renewable and conventional
G_erv = G[occursin.("Wind",G) .| occursin.("Solar",G)];
G_conv = G[G.∉ Ref(G_erv)];

# We want to select the columns that do work for us. We will remove those in kW
columns = names(generators)
to_use = columns[.!occursin.("KW",columns) .& .!occursin.("Gen",columns)]

gen_data = Dict()
for column in to_use
    colname = split(column)[1]
    gen_data[colname] = JuMP.Containers.DenseAxisArray(generators[!,column],G)
end

In [31]:
demand = CSV.read("Data/demanddata_Hourly.csv",DataFrame);
D = demand.ND;

In [32]:
H = 1:length(D)

1:8760

In [33]:
cap_df = CSV.read("Data/TMY_uk_erv.csv",DataFrame);
names_x = names(cap_df);
for gen in G_erv
    if occursin("Solar",gen)
        tech = "solar"
    else
        tech = lowercase(split(gen,"W")[1])
    end
    for name in names_x
        if occursin(tech,name)
            rename!(cap_df, name => gen)
            names_x = names(cap_df)
        end
    end

end

cap_df = cap_df[!,G_erv];
capacity_factors = Dict()
for column in names(cap_df)
   capacity_factors[column] = JuMP.Containers.DenseAxisArray(cap_df[!,column],H)
end

VoLL = 1000;

In [34]:
FirstExpansionModel = Model(Gurobi.Optimizer)
@variables(FirstExpansionModel,begin
    CAP[g in G] >= 0
    GEN[g in G, h in H] >= 0
    NSE[h in H] >= 0
end);

@constraints(FirstExpansionModel,begin
    demandBalance[h in H], sum(GEN[g,h] for g in G) + NSE[h] == D[h]
    capacityBound[g in G_conv, h in H], GEN[g,h] <= CAP[g]*gen_data["CapacityFactor"][g]
    capacityBoundRVE[g in G_erv, h in H], GEN[g,h] <= CAP[g]*capacity_factors[g][h]
end)

@objective(FirstExpansionModel,Min,sum((gen_data["CAPEX"][g]+gen_data["OM"][g])*CAP[g] for g in G) + sum((gen_data["FuelCost"][g]*gen_data["HeatRate"][g]+gen_data["VarOM"][g])*GEN[g,h] for g in G, h in H) + sum(VoLL*NSE[h] for h in H))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-12


123739.12 CAP[OnshoreWind] + 401212.37 CAP[OffshoreWind] + 106981.4 CAP[Solar] + 704816.38 CAP[Hydro-Dam] + 570804.12 CAP[Hydro-RoR] + 771658.4 CAP[Nuclear] + 507692.29 CAP[Biomass] + 539800.14 CAP[Coal] + 113290.07 CAP[Gas-CC] + 96857.61 CAP[Gas-CT] + 9.40426 GEN[Nuclear,1] + 9.40426 GEN[Nuclear,2] + 9.40426 GEN[Nuclear,3] + 9.40426 GEN[Nuclear,4] + 9.40426 GEN[Nuclear,5] + 9.40426 GEN[Nuclear,6] + 9.40426 GEN[Nuclear,7] + 9.40426 GEN[Nuclear,8] + 9.40426 GEN[Nuclear,9] + 9.40426 GEN[Nuclear,10] + 9.40426 GEN[Nuclear,11] + 9.40426 GEN[Nuclear,12] + 9.40426 GEN[Nuclear,13] + 9.40426 GEN[Nuclear,14] + 9.40426 GEN[Nuclear,15] + 9.40426 GEN[Nuclear,16] + 9.40426 GEN[Nuclear,17] + 9.40426 GEN[Nuclear,18] + 9.40426 GEN[Nuclear,19] + 9.40426 GEN[Nuclear,20] + [[...52510 terms omitted...]] + 1000 NSE[8731] + 1000 NSE[8732] + 1000 NSE[8733] + 1000 NSE[8734] + 1000 NSE[8735] + 1000 NSE[8736] + 1000 NSE[8737] + 1000 NSE[8738] + 1000 NSE[8739] + 1000 NSE[8740] + 1000 NSE[8741] + 1000 NSE[8742] + 

In [35]:
maximum(D)

87809

In [36]:
optimize!(FirstExpansionModel)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 96360 rows, 96370 columns and 267962 nonzeros
Model fingerprint: 0x39e9bdb8
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  Objective range  [9e+00, 8e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+04, 9e+04]
Presolve removed 3608 rows and 3608 columns
Presolve time: 0.13s
Presolved: 92752 rows, 92762 columns, 260738 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.02s

Barrier statistics:
 Dense cols : 10
 AA' NZ     : 1.680e+05
 Factor NZ  : 1.430e+06 (roughly 90 MB of memory)
 Factor Ops : 2.296e+07 (less than 1 second per iteration)
 Threads    : 10

                  Objective                Residual
Iter       Primal   

In [37]:
generation = value.(GEN).data*ones(8760,1)
total_generation = JuMP.Containers.DenseAxisArray(generation[:],G)
total_co2 = sum(gen_data["CO2"][g]*gen_data["HeatRate"][g]*total_generation[g] for g in G)

3.45021630269308e11

In [38]:
SecondExpansionModel = Model(Gurobi.Optimizer)
@variables(SecondExpansionModel,begin
    CAP[g in G] >= 0
    GEN[g in G, h in H] >= 0
    NSE[h in H] >= 0
end);

@constraints(SecondExpansionModel,begin
    demandBalance[h in H], sum(GEN[g,h] for g in G) + NSE[h] == D[h]
    capacityBound[g in G_conv, h in H], GEN[g,h] <= CAP[g]*gen_data["CapacityFactor"][g]
    capacityBoundRVE[g in G_erv, h in H], GEN[g,h] <= CAP[g]*capacity_factors[g][h]
    CO2Constraint, sum(gen_data["CO2"][g]*gen_data["HeatRate"][g]*GEN[g,h] for g in G, h in H) <= total_co2 * 0.5
end)

@objective(SecondExpansionModel,Min,sum((gen_data["CAPEX"][g]+gen_data["OM"][g])*CAP[g] for g in G) + sum((gen_data["FuelCost"][g]*gen_data["HeatRate"][g]+gen_data["VarOM"][g])*GEN[g,h] for g in G, h in H) + sum(VoLL*NSE[h] for h in H))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-12


123739.12 CAP[OnshoreWind] + 401212.37 CAP[OffshoreWind] + 106981.4 CAP[Solar] + 704816.38 CAP[Hydro-Dam] + 570804.12 CAP[Hydro-RoR] + 771658.4 CAP[Nuclear] + 507692.29 CAP[Biomass] + 539800.14 CAP[Coal] + 113290.07 CAP[Gas-CC] + 96857.61 CAP[Gas-CT] + 9.40426 GEN[Nuclear,1] + 9.40426 GEN[Nuclear,2] + 9.40426 GEN[Nuclear,3] + 9.40426 GEN[Nuclear,4] + 9.40426 GEN[Nuclear,5] + 9.40426 GEN[Nuclear,6] + 9.40426 GEN[Nuclear,7] + 9.40426 GEN[Nuclear,8] + 9.40426 GEN[Nuclear,9] + 9.40426 GEN[Nuclear,10] + 9.40426 GEN[Nuclear,11] + 9.40426 GEN[Nuclear,12] + 9.40426 GEN[Nuclear,13] + 9.40426 GEN[Nuclear,14] + 9.40426 GEN[Nuclear,15] + 9.40426 GEN[Nuclear,16] + 9.40426 GEN[Nuclear,17] + 9.40426 GEN[Nuclear,18] + 9.40426 GEN[Nuclear,19] + 9.40426 GEN[Nuclear,20] + [[...52510 terms omitted...]] + 1000 NSE[8731] + 1000 NSE[8732] + 1000 NSE[8733] + 1000 NSE[8734] + 1000 NSE[8735] + 1000 NSE[8736] + 1000 NSE[8737] + 1000 NSE[8738] + 1000 NSE[8739] + 1000 NSE[8740] + 1000 NSE[8741] + 1000 NSE[8742] + 

In [50]:
optimize!(SecondExpansionModel)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 96361 rows, 96370 columns and 294242 nonzeros
Coefficient statistics:
  Matrix range     [1e-04, 2e+03]
  Objective range  [9e+00, 8e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+04, 2e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.340198409e+10

User-callback calls 18, time in user-callback 0.00 sec


In [52]:
display(value.(CAP))
CO2price = abs(dual.(CO2Constraint));

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, String15["OnshoreWind", "OffshoreWind", "Solar", "Hydro-Dam", "Hydro-RoR", "Nuclear", "Biomass", "Coal", "Gas-CC", "Gas-CT"]
And data, a 10-element Vector{Float64}:
 105753.65344467641
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
  53108.122755741126
  10113.173277661797

In [45]:
ThirdExpansionModel = Model(Gurobi.Optimizer)
@variables(ThirdExpansionModel,begin
    CAP[g in G] >= 0
    GEN[g in G, h in H] >= 0
    NSE[h in H] >= 0
    CO2[g in G] >= 0
end);

@constraints(ThirdExpansionModel,begin
    demandBalance[h in H], sum(GEN[g,h] for g in G) + NSE[h] == D[h]
    capacityBound[g in G_conv, h in H], GEN[g,h] <= CAP[g]*gen_data["CapacityFactor"][g]
    capacityBoundRVE[g in G_erv, h in H], GEN[g,h] <= CAP[g]*capacity_factors[g][h]
    #CO2Constraint, sum(gen_data["CO2"][g]*gen_data["HeatRate"][g]*GEN[g,h] for g in G, h in H) <= total_co2 * 0.5
    CO2Emissions[g in G], CO2[g] >= sum(gen_data["CO2"][g]*gen_data["HeatRate"][g]*GEN[g,h] for h in H)
end)

@objective(ThirdExpansionModel,Min,sum((gen_data["CAPEX"][g]+gen_data["OM"][g])*CAP[g] for g in G) + sum((gen_data["FuelCost"][g]*gen_data["HeatRate"][g]+gen_data["VarOM"][g])*GEN[g,h] for g in G, h in H) + sum(VoLL*NSE[h] for h in H)
+ sum(CO2price*CO2[g] for g in G))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-12


123739.12 CAP[OnshoreWind] + 401212.37 CAP[OffshoreWind] + 106981.4 CAP[Solar] + 704816.38 CAP[Hydro-Dam] + 570804.12 CAP[Hydro-RoR] + 771658.4 CAP[Nuclear] + 507692.29 CAP[Biomass] + 539800.14 CAP[Coal] + 113290.07 CAP[Gas-CC] + 96857.61 CAP[Gas-CT] + 9.40426 GEN[Nuclear,1] + 9.40426 GEN[Nuclear,2] + 9.40426 GEN[Nuclear,3] + 9.40426 GEN[Nuclear,4] + 9.40426 GEN[Nuclear,5] + 9.40426 GEN[Nuclear,6] + 9.40426 GEN[Nuclear,7] + 9.40426 GEN[Nuclear,8] + 9.40426 GEN[Nuclear,9] + 9.40426 GEN[Nuclear,10] + 9.40426 GEN[Nuclear,11] + 9.40426 GEN[Nuclear,12] + 9.40426 GEN[Nuclear,13] + 9.40426 GEN[Nuclear,14] + 9.40426 GEN[Nuclear,15] + 9.40426 GEN[Nuclear,16] + 9.40426 GEN[Nuclear,17] + 9.40426 GEN[Nuclear,18] + 9.40426 GEN[Nuclear,19] + 9.40426 GEN[Nuclear,20] + [[...52520 terms omitted...]] + 1000 NSE[8741] + 1000 NSE[8742] + 1000 NSE[8743] + 1000 NSE[8744] + 1000 NSE[8745] + 1000 NSE[8746] + 1000 NSE[8747] + 1000 NSE[8748] + 1000 NSE[8749] + 1000 NSE[8750] + 1000 NSE[8751] + 1000 NSE[8752] + 

In [53]:
optimize!(ThirdExpansionModel)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 96370 rows, 96380 columns and 294252 nonzeros
Coefficient statistics:
  Matrix range     [1e-04, 2e+03]
  Objective range  [6e-02, 8e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+04, 9e+04]

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.443292588e+10

User-callback calls 15, time in user-callback 0.00 sec


In [54]:
value.(CAP)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, String15["OnshoreWind", "OffshoreWind", "Solar", "Hydro-Dam", "Hydro-RoR", "Nuclear", "Biomass", "Coal", "Gas-CC", "Gas-CT"]
And data, a 10-element Vector{Float64}:
 105753.65344467641
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
  53108.122755741126
  10113.173277661797